# Import libs

In [1]:
!python --version

Python 3.10.13


In [2]:
import catboost
from catboost import CatBoostRanker, Pool
from copy import deepcopy
import numpy as np
import os
import pandas as pd

In [3]:
print(f" pandas version {pd.__version__}")
print(f" numpy version {np.__version__}")
print(f" catboost version {catboost.__version__}")

 pandas version 2.2.0
 numpy version 1.26.4
 catboost version 1.2.2


In [4]:
#!pip install numpy==1.26.4
#!pip install pandas==2.2.0
#!pip install catboost==1.2.2

# Data preprocessing

## Read and check data

In [70]:
train_df = pd.read_csv('/kaggle/input/vk-task/train_df.csv')
test_df = pd.read_csv('/kaggle/input/vk-task/test_df.csv')

In [18]:
train_df.head()

,search_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,target
0,758,9,0,0,1,20,3,40,0,3,...,0.204682,0.271755,0.055623,0,0,0,0.38648,0.0,0.0,0
1,758,9,0,0,1,20,3,40,0,3,...,0.195531,0.188787,0.036914,0,0,0,0.10982,0.0,0.0,0
2,758,9,0,0,1,20,3,40,0,3,...,0.148609,0.186517,0.027718,0,0,0,0.03674,0.0,0.0,0
3,758,9,0,0,1,20,3,40,0,3,...,0.223748,0.229039,0.051247,0,0,0,0.00000,0.0,0.0,0
4,758,9,0,0,1,20,3,40,0,3,...,0.170935,0.249031,0.042568,0,0,0,0.00000,0.0,0.0,0


In [19]:
train_df.describe()

,search_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,target
count,15081.000000,15081.0,15081.000000,15081.000000,15081.000000,15081.000000,15081.000000,15081.000000,15081.000000,15081.000000,...,15081.000000,15081.000000,15081.000000,15081.0,15081.0,15081.0,15081.000000,15081.000000,15081.000000,15081.000000
mean,254806.827465,9.0,0.193754,0.464293,0.629865,15.928519,3.876401,33.802467,1.211193,1.793847,...,0.199545,0.426543,0.096461,0.0,0.0,0.0,0.297465,0.030543,0.031207,0.021351
std,142374.568081,0.0,0.626654,1.183076,0.482857,5.311422,0.469749,11.316810,2.234887,1.885131,...,0.088724,0.247388,0.063323,0.0,0.0,0.0,0.311962,0.112984,0.115598,0.144558
min,116.000000,9.0,0.000000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
25%,134823.000000,9.0,0.000000,0.000000,0.000000,9.000000,4.000000,26.000000,0.000000,0.000000,...,0.168583,0.255086,0.052658,0.0,0.0,0.0,0.036740,0.000000,0.000000,0.000000
50%,255859.000000,9.0,0.000000,0.000000,1.000000,20.000000,4.000000,34.000000,0.000000,2.000000,...,0.212743,0.430880,0.091705,0.0,0.0,0.0,0.176140,0.000000,0.000000,0.000000
75%,376175.000000,9.0,0.000000,0.000000,1.000000,20.000000,4.000000,44.000000,1.000000,3.000000,...,0.253825,0.603686,0.136017,0.0,0.0,0.0,0.529100,0.000000,0.000000,0.000000
max,496975.000000,9.0,3.000000,6.000000,1.000000,20.000000,6.000000,49.000000,7.000000,7.000000,...,0.485510,0.999072,0.411563,0.0,0.0,0.0,1.000020,0.846272,0.908132,1.000000


In [20]:
test_df.describe()

,search_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,target
count,1529.000000,1529.0,1529.000000,1529.000000,1529.000000,1529.000000,1529.000000,1529.000000,1529.000000,1529.000000,...,1529.000000,1529.000000,1529.000000,1529.0,1529.0,1529.0,1529.000000,1529.000000,1529.000000,1529.000000
mean,275783.351864,9.0,0.176586,0.452583,0.614127,15.755396,3.858731,32.606933,1.057554,1.833878,...,0.205209,0.448991,0.103061,0.0,0.0,0.0,0.297876,0.030655,0.031390,0.022237
std,149946.991689,0.0,0.557153,1.117101,0.486960,5.356561,0.399186,10.926778,2.075937,1.969301,...,0.086952,0.255577,0.067747,0.0,0.0,0.0,0.316741,0.111981,0.113095,0.147501
min,10655.000000,9.0,0.000000,0.000000,0.000000,9.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
25%,123542.000000,9.0,0.000000,0.000000,0.000000,9.000000,4.000000,25.000000,0.000000,0.000000,...,0.176786,0.267725,0.055150,0.0,0.0,0.0,0.036740,0.000000,0.000000,0.000000
50%,305790.000000,9.0,0.000000,0.000000,1.000000,20.000000,4.000000,31.000000,0.000000,1.000000,...,0.216342,0.488493,0.101199,0.0,0.0,0.0,0.145400,0.000000,0.000000,0.000000
75%,397875.000000,9.0,0.000000,0.000000,1.000000,20.000000,4.000000,44.000000,0.000000,3.000000,...,0.256027,0.628704,0.143651,0.0,0.0,0.0,0.536740,0.000000,0.000000,0.000000
max,494635.000000,9.0,2.000000,5.000000,1.000000,20.000000,4.000000,49.000000,7.000000,6.000000,...,0.445427,0.997673,0.394140,0.0,0.0,0.0,1.000000,0.755875,0.672784,1.000000


## Split data

In [21]:
X_train = train_df.drop(['search_id', 'target'], axis=1).values
y_train = train_df['target'].values
queries_train = train_df['search_id'].values

X_test = test_df.drop(['search_id', 'target'], axis=1).values
y_test = test_df['target'].values
queries_test = test_df['search_id'].values

In [22]:
print(X_train.shape, X_test.shape)

(15081, 79) (1529, 79)


In [23]:
train_pool = Pool(
    data=X_train,
    label=y_train,
    group_id=queries_train
)

test_pool = Pool(
    data=X_test,
    label=y_test,
    group_id=queries_test
)


def create_weights(queries):
    query_set = np.unique(queries)
    query_weights = np.random.uniform(size=query_set.shape[0])
    weights = np.zeros(shape=queries.shape)
    for i, query_id in enumerate(query_set):
        weights[queries == query_id] = query_weights[i]
    return weights
    

train_with_weights = Pool(
    data=X_train,
    label=y_train,
    group_weight=create_weights(queries_train),
    group_id=queries_train
)

test_with_weights = Pool(
    data=X_test,
    label=y_test,
    group_weight=create_weights(queries_test),
    group_id=queries_test
)

## Model and parameters

In [41]:
default_parameters = {
    'iterations': 500,
    'custom_metric': ['NDCG'],
    'verbose': False,
    'random_seed': 42,
    "use_best_model":True,
}

def fit_model(loss_function, additional_params=None, train_pool=train_pool, test_pool=test_pool):
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function
    
    if additional_params is not None:
        parameters.update(additional_params)
        
    model = CatBoostRanker(**parameters)
    model.fit(train_pool, eval_set=test_pool)
    return model

## Train models with different loss function

In [42]:
model_rmse = fit_model('RMSE')
print(model_rmse.get_best_score()['validation']['NDCG:type=Base'])

0.9265243609418718


In [43]:
model_rmse_with_weights = fit_model(
    'RMSE', 
    additional_params={'train_dir': 'RMSE_weigths'}, 
    train_pool=train_with_weights,
    test_pool=test_with_weights
)
print(model_rmse_with_weights.get_best_score()['validation']['NDCG:type=Base'])

0.9132983953185286


In [44]:
model_query_rmse= fit_model('QueryRMSE')
print(model_query_rmse.get_best_score()['validation']['NDCG:type=Base'])

0.9337307340228201


In [45]:
model_pair_logit = fit_model('PairLogit')
print(model_pair_logit.get_best_score()['validation']['NDCG:type=Base'])

0.9324564569315654


In [46]:
model_pair_logit_pairwise = fit_model('PairLogitPairwise')
print(model_pair_logit_pairwise.get_best_score()['validation']['NDCG:type=Base'])

0.9201702162974178


In [47]:
model_yeti_rank = fit_model(
    'YetiRank', 
    additional_params={'custom_metric': ['NDCG', 'PFound', 'AverageGain:top=10']}, 
)
print(model_yeti_rank.get_best_score()['validation']['NDCG:type=Base'])

0.9264108881207633


In [48]:
model_yeti_rank_pairwise = fit_model(
    'YetiRankPairwise', 
    additional_params={'custom_metric': ['NDCG', 'PFound', 'AverageGain:top=10']}, 
)
print(model_yeti_rank_pairwise.get_best_score()['validation']['NDCG:type=Base'])

0.9253106939759423


# Summary

Был выбран CatBoostRanker как один из методов для решения данной задачи, который подходит для таких задач с табличными данными.

Данные были прочитаны, проверены на Nan и специфичные значения полей, которые были не обнаружены. Дополнительные данные(новые фичи) было невозможно создать, т.к. фичи были без именнований и другой дополнительной информации.

Модели уже имеют хороший скор и на 20 итерациях, но я запускал на 500 и выбирал лучшую модель по скору.

Лучший скор был 0,933, что является достаточно высоким фактором, т.к. диапозон значения метрики NВСG [0, 1].

Дополнительное деление на train/val не было, т.к. есть тестовые данные, которые открыты и на них надо выбить скор, но при закрытых данных использовал бы метод кросс-валидации